In [ ]:
from utils._model import build_and_compile_model, train_model_with_kfold, train_models_with_kfold, display_summaries
from keras.layers import Dropout, Dense, GlobalAveragePooling2D, Flatten, LeakyReLU, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LambdaCallback
from tensorflow.keras.applications import EfficientNetB0, DenseNet121, MobileNetV2, ResNet50
from sklearn.metrics import confusion_matrix, classification_report
from utils._evaluation import evaluate_models, generate_reports, generate_single_report
from keras.preprocessing.image import ImageDataGenerator
from utils._image import display_images_from_subfolders
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from utils._data import get_subfolders, load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from keras.initializers import GlorotUniform
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from keras.models import Model
from keras.metrics import AUC
from utils._plot import plot
from tensorflow import keras
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import cv2
import os
import io

In [ ]:
labels = ['glioma', 'meningioma', 'no_tumor', 'pituitary']
testing_dir = 'Testing/'
training_dir = 'Training/'

In [ ]:
subfolders = get_subfolders(training_dir)
display_images_from_subfolders(training_dir, subfolders)

## Creating training and testing data frames

In [ ]:
X_train, y_train = load_data(training_dir, labels)
X_test, y_test = load_data(testing_dir, labels)

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)

y_train = to_categorical(y_train) # One-hot encoding of integer labels
y_test = to_categorical(y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) # 20% of the data is validation data

In [ ]:
image_gen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

image_gen.fit(X_train)

In [ ]:
models = {
    'EfficientNet': build_and_compile_model(architecture='EfficientNet', image_size=224),
    'DenseNet': build_and_compile_model(architecture='DenseNet', image_size=224),
    'MobileNet': build_and_compile_model(architecture='MobileNet', image_size=224),
    'ResNet50': build_and_compile_model(architecture='ResNet50', image_size=224)
}

display_summaries(models)

## K-Fold Cross Validation

Instead of training each model once on a static train/validation split, each model will now be trained and evaluated n_splits times. Each time it will be with a different train/validation set derived from the original X_train and y_train. In other words, we split the data into k subsets and validate the model k times.

In [ ]:
all_metrics = train_models_with_kfold(models, X_train, y_train, image_gen)

evaluate_models(models, X_test, y_test)
generate_reports(all_metrics, X_test, y_test, models)

## Create custom Convolutional Neural Network

In [ ]:
model = Sequential([
    # First Conv Block
    Conv2D(32, (3, 3), kernel_initializer='he_normal', input_shape=(224, 224, 3)),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),
    
    # Second Conv Block
    Conv2D(64, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),
    
    # Third Conv Block
    Conv2D(128, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),
    
    # New Fourth Conv Block
    Conv2D(256, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),

    # New Fifth Conv Block
    Conv2D(512, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),

    # Flatten and Fully Connected Layers
    Flatten(),
    Dense(128, kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.5),
    
    Dense(64, kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.5),

    # Output Layer
    Dense(4, activation='softmax', kernel_regularizer=l2(0.01))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])
model.summary()


In [ ]:
metrics = train_model_with_kfold(model, 'CustomNet', X_train, y_train, image_gen, n_splits=5)
generate_single_report('CustomNet', metrics, model, X_test, y_test)